[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/recommendation/product-recommender/product_recommender.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/recommendation/product-recommender/product_recommender.ipynb)

# Product Recommendation Engine

Learn how to build a product recommendation engine using collaborative filtering and Pinecone.

In this example, we will generate product recommendations for ecommerce customers based on previous orders and trending items. This example covers preparing the vector embeddings, creating and deploying the Pinecone service, writing data to Pinecone, and finally querying Pinecone to receive a ranked list of recommended products.

---

🚨 _Note that running this on CPU is slow! If running on Google Colab you go to **Runtime > Change runtime type > Hardware accelerator > GPU** to switch to GPU._

---

## Data Preparation

In [1]:
!pip install -qU numpy pandas scipy

You should consider upgrading via the '/home/jelena/.pyenv/versions/3.8.16/envs/pinecone-product-recommender/bin/python3.8 -m pip install --upgrade pip' command.


**Import Python Libraries**

In [2]:
import os
import time
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import itertools

In [3]:
!pip install -qU kaggle

You should consider upgrading via the '/home/jelena/.pyenv/versions/3.8.16/envs/pinecone-product-recommender/bin/python3.8 -m pip install --upgrade pip' command.


In [4]:
try:
    import kaggle
except OSError as e:
    print(e)

The first time you `import kaggle` you will see an `OSError`, that is because we need to add our Kaggle credentials to the `/root/.kaggle/kaggle.json` file. You can find these credentials on [Kaggle](https://kaggle.com) by accessing your profile in the top-right corner of the page. This will download a `kaggle.json` file which contains your username and secret key. You can enter them below:

In [5]:
import json

with open('/root/.kaggle/kaggle.json', 'w') as fp:
    fp.write(json.dumps({"username":"YOUR_USERNAME","key":"YOUR_SECRET_KEY"}))

Now we can download the dataset:

In [6]:
!kaggle competitions download -c instacart-market-basket-analysis

100%|███████████████████████████████████████▉| 196M/196M [00:27<00:00, 9.01MB/s]
100%|████████████████████████████████████████| 196M/196M [00:27<00:00, 7.36MB/s]


This downloads a set of zip files, we extract them like so:

In [7]:
import zipfile

files = [
    'instacart-market-basket-analysis.zip',
    'order_products__train.csv.zip',
    'order_products__prior.csv.zip',
    'products.csv.zip',
    'orders.csv.zip'
]

for filename in files:
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('./')

Now we can move on to loading the dataset.

**Load the (Example) Instacart Data**

We are going to use the [Instacart Market Basket Analysis](https://www.kaggle.com/c/instacart-market-basket-analysis/data) dataset for this task.

The data used throughout this example is a set of files describing customers' orders over time. The main focus is on the *orders.csv* file, where each line represents a relation between a user and the order. In other words, each line has information on *userid* (user who made the order) and *orderid*. Note there is no information about products in this table. Product information related to specific orders is stored in the *order_product__*.csv* dataset.

In [8]:
order_products_train = pd.read_csv('order_products__train.csv')
order_products_prior = pd.read_csv('order_products__prior.csv')
products = pd.read_csv('products.csv')
orders = pd.read_csv('orders.csv')

order_products = pd.concat([order_products_train, order_products_prior])

**Preparing data for the model**


The Collaborative Filtering model used in this example requires only users’ historical preferences on a set of items. As there is no explicit rating in the data we are using, the purchase quantity can represent a “confidence” in terms of how strong the interaction was between the user and the products.

The dataframe data will store this data and will be the base for the model.

In [9]:
customer_order_products = pd.merge(orders, order_products, how='inner',on='order_id')

# creating a table with "confidences"
data = customer_order_products.groupby(['user_id', 'product_id'])[['order_id']].count().reset_index()
data.columns=["user_id", "product_id", "total_orders"]
data.product_id = data.product_id.astype('int64')

# Create a lookup frame so we can get the product names back in readable form later.
products_lookup = products[['product_id', 'product_name']].drop_duplicates()
products_lookup['product_id'] = products_lookup.product_id.astype('int64')

We will create two prototype users here and add them to our data dataframe. Each user will be buying only a specific product:
- The first user will be buying only **Mineral Water**
- The second user will be buying baby products: **No More Tears Baby Shampoo** and **Baby Wash & Shampoo**

These users will be later used for querying and examination of the model results.

In [10]:
data_new = pd.DataFrame([[data.user_id.max() + 1, 22802, 97],
                         [data.user_id.max() + 2, 26834, 89],
                         [data.user_id.max() + 2, 12590, 77]
                        ], columns=['user_id', 'product_id', 'total_orders'])
data_new

,user_id,product_id,total_orders
0,206210,22802,97
1,206211,26834,89
2,206211,12590,77


In [11]:
data = pd.concat([data, data_new]).reset_index(drop = True)
data.tail()

,user_id,product_id,total_orders
13863744,206209,48697,1
13863745,206209,48742,2
13863746,206210,22802,97
13863747,206211,26834,89
13863748,206211,12590,77


In the next step, we will first extract user and item unique ids, in order to create a CSR (Compressed Sparse Row) matrix. 


In [12]:
users = list(np.sort(data.user_id.unique()))
items = list(np.sort(products.product_id.unique()))
purchases = list(data.total_orders)

# create zero-based index position <-> user/item ID mappings
index_to_user = pd.Series(users)

# create reverse mappings from user/item ID to index positions
user_to_index = pd.Series(data=index_to_user.index + 1, index=index_to_user.values)

# create zero-based index position <-> item/user ID mappings
index_to_item = pd.Series(items)

# create reverse mapping from item/user ID to index positions
item_to_index = pd.Series(data=index_to_item.index, index=index_to_item.values)

# Get the rows and columns for our new matrix
products_rows = data.product_id.astype(int)
users_cols = data.user_id.astype(int)

# Create a sparse matrix for our users and products containing number of purchases
sparse_product_user = sparse.csr_matrix((purchases, (products_rows, users_cols)), shape=(len(items) + 1, len(users) + 1))
sparse_product_user.data = np.nan_to_num(sparse_product_user.data, copy=False)

sparse_user_product = sparse.csr_matrix((purchases, (users_cols, products_rows)), shape=(len(users) + 1, len(items) + 1))
sparse_user_product.data = np.nan_to_num(sparse_user_product.data, copy=False)

## Implicit Model

In this section we will demonstrate creation and training of a recommender model using the **implicit** library. The recommendation model is based off the algorithms described in the paper [Collaborative Filtering for Implicit Feedback Datasets](https://www.researchgate.net/publication/220765111_Collaborative_Filtering_for_Implicit_Feedback_Datasets) with performance optimizations described in [Applications of the Conjugate Gradient Method for Implicit Feedback Collaborative Filtering](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.379.6473&rep=rep1&type=pdf).


In [13]:
!pip install -qU implicit

You should consider upgrading via the '/home/jelena/.pyenv/versions/3.8.16/envs/pinecone-product-recommender/bin/python3.8 -m pip install --upgrade pip' command.


In [14]:
import implicit
from implicit import evaluation

#split data into train and test sets
train_set, test_set = evaluation.train_test_split(sparse_user_product, train_percentage=0.9)

# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=100,
                                             regularization=0.05,
                                             iterations=50,
                                             num_threads=1)

alpha_val = 15
train_set = (train_set * alpha_val).astype('double')

# train the model on a sparse matrix of item/user/confidence weights
model.fit(train_set, show_progress = True)

/home/jelena/.pyenv/versions/3.8.16/envs/pinecone-product-recommender/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 50/50 [12:30<00:00, 15.02s/it]


We will evaluate the model using the inbuilt library function

In [15]:
test_set = (test_set * alpha_val).astype('double')
evaluation.ranking_metrics_at_k(model, train_set, test_set, K=100,
                         show_progress=True, num_threads=1)

100%|██████████| 193027/193027 [01:22<00:00, 2331.67it/s]


{'precision': 0.2749977629022807,
 'map': 0.04447513449736459,
 'ndcg': 0.1442493819257263,
 'auc': 0.6550438975250414}

This is what item and user factors look like. These vectors will be stored in our vector index later and used for recommendation.

In [16]:
model.item_factors[1:3]

array([[ 7.62302289e-03, -1.87499879e-03, -2.67132535e-04,
        -1.98393618e-03,  1.25893755e-02,  2.26164684e-02,
        -2.98452517e-03,  1.48304272e-03,  4.13032155e-03,
         1.40623152e-02,  7.02867238e-03,  1.75751895e-02,
        -8.48765578e-03,  7.23443553e-03,  1.19221059e-03,
         2.13307124e-02,  1.94959883e-02,  5.55894384e-03,
         5.24600095e-04,  4.55727894e-03,  8.32605269e-03,
         2.58112140e-03, -4.82219504e-04, -1.20439762e-02,
         2.47381651e-03,  1.28009180e-02,  2.08530622e-03,
        -3.33145494e-03,  1.92555189e-02,  4.34594555e-03,
        -6.63968595e-03,  5.56971226e-03,  6.72743656e-03,
         2.78714020e-03, -2.83625396e-03, -6.03283104e-03,
         8.45496822e-03,  1.18931178e-02,  2.10034065e-02,
        -1.68929249e-02,  1.79267656e-02, -9.86911077e-03,
         1.26773408e-02, -1.57724898e-02,  8.18107463e-03,
        -3.72606446e-03, -1.64890327e-02,  1.08041009e-02,
         1.01122679e-02,  1.74866407e-03, -1.00692408e-0

In [17]:
model.user_factors[1:3]

array([[-1.1535235e-01, -1.0716660e+00,  1.9001467e+00, -2.8103048e-01,
         2.3507787e-01, -1.3524084e-01, -9.7265404e-01, -2.4441715e-01,
        -1.3157841e+00, -9.3033111e-01,  1.3205792e+00,  1.5934669e+00,
        -8.7007034e-01,  8.2152611e-01, -1.1706644e+00,  1.0730673e+00,
         1.8356223e+00, -6.4666368e-02, -3.6772355e-01,  9.4636500e-01,
         2.3708563e+00, -1.0900316e+00, -1.9363825e-01, -1.2721621e+00,
        -3.8839546e-01,  1.5143086e+00, -8.0761343e-01, -9.5621413e-01,
         1.4447758e+00, -3.5387911e-02,  1.3933902e+00,  8.5588759e-01,
         4.9890396e-01,  3.4040824e-01, -3.7110674e-01,  3.9586887e-01,
        -4.4984985e-02,  1.0853641e+00,  1.0416827e+00, -2.0454068e+00,
         2.1352434e+00, -1.4058716e+00,  8.2302362e-01,  2.8762394e-01,
        -8.2267590e-02, -9.1558552e-01,  5.0190097e-01,  2.0039406e-01,
         1.2773964e+00, -4.9887997e-01, -1.5381695e-02, -7.4103689e-01,
        -4.8554859e-01, -6.2013620e-01,  1.2602403e+00,  3.72646

## Configure Pinecone

Install and setup Pinecone

In [18]:
!pip install -qU pinecone-client

You should consider upgrading via the '/home/jelena/.pyenv/versions/3.8.16/envs/pinecone-product-recommender/bin/python3.8 -m pip install --upgrade pip' command.


In [19]:
import pinecone

In [20]:
# Load Pinecone API key
api_key = os.getenv('PINECONE_API_KEY') or 'YOUR_API_KEY'
# Set Pinecone environment. Find next to API key in console
env = os.getenv('PINECONE_ENVIRONMENT') or "YOUR_ENV"

pinecone.init(api_key=api_key, environment=env)

[Get a Pinecone API key](http://app.pinecone.io/) if you don't have one.

In [36]:
#List all present indexes associated with your key, should be empty on the first run
pinecone.list_indexes()

[]

**Create an Index**

In [37]:
# Set a name for your index
index_name = 'product-recommender'

In [38]:
# Make sure service with the same name does not exist
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)
pinecone.create_index(name=index_name, dimension=100)

**Connect to the new index**

In [39]:
index = pinecone.Index(index_name=index_name)

## Load Data

Uploading all items (products that one can buy) and displaying some examples of products and their vector representations.


In [40]:
!pip install -qU torch

You should consider upgrading via the '/home/jelena/.pyenv/versions/3.8.16/envs/pinecone-product-recommender/bin/python3.8 -m pip install --upgrade pip' command.


In [41]:
import torch

# Get all of the items
all_items_titles = [{'title': title} for title in products_lookup['product_name']]
all_items_ids = [str(product_id) for product_id in products_lookup['product_id']]

# Transform items into factors
items_factors = model.item_factors

device = "cuda" if torch.cuda.is_available() else "cpu"
item_embeddings = items_factors[1:].to_numpy().tolist() if device == "cuda" else items_factors[1:].tolist()

# Prepare item factors for upload
items_to_insert = list(zip(all_items_ids, item_embeddings, all_items_titles))
display(items_to_insert[:2])

[('1',
  [0.007623022887855768,
   -0.0018749987939372659,
   -0.0002671325346454978,
   -0.0019839361775666475,
   0.012589375488460064,
   0.022616468369960785,
   -0.0029845251701772213,
   0.001483042724430561,
   0.004130321554839611,
   0.014062315225601196,
   0.007028672378510237,
   0.017575189471244812,
   -0.008487655781209469,
   0.007234435528516769,
   0.0011922105913981795,
   0.021330712363123894,
   0.01949598826467991,
   0.0055589438416063786,
   0.0005246000946499407,
   0.00455727893859148,
   0.008326052688062191,
   0.002581121399998665,
   -0.0004822195041924715,
   -0.012043976224958897,
   0.0024738165084272623,
   0.012800917960703373,
   0.002085306216031313,
   -0.003331454936414957,
   0.019255518913269043,
   0.004345945548266172,
   -0.006639685947448015,
   0.00556971225887537,
   0.0067274365574121475,
   0.0027871401980519295,
   -0.002836253959685564,
   -0.006032831035554409,
   0.008454968221485615,
   0.011893117800354958,
   0.02100340649485588,


**Insert items into the index**

In [42]:
from tqdm.auto import tqdm

BATCH_SIZE = 100

print('Index statistics before upsert:', index.describe_index_stats())

for i in tqdm(range(0, len(items_to_insert), BATCH_SIZE)):
        index.upsert(vectors=items_to_insert[i:i+BATCH_SIZE])

print('Index statistics after upsert:', index.describe_index_stats())

Index statistics before upsert: {'dimension': 100,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


100%|██████████| 497/497 [05:36<00:00,  1.48it/s]


Index statistics after upsert: {'dimension': 100,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 49688}},
 'total_vector_count': 49688}


This is a helper method for analysing recommendations later.
This method returns top N products that someone bought in the past (based on product quantity).

In [43]:
def products_bought_by_user_in_the_past(user_id: int, top: int = 10):

    selected = data[data.user_id == user_id].sort_values(by=['total_orders'], ascending=False)

    selected['product_name'] = selected['product_id'].map(products_lookup.set_index('product_id')['product_name'])
    selected = selected[['product_id', 'product_name', 'total_orders']].reset_index(drop=True)
    if selected.shape[0] < top:
        return selected

    return selected[:top]

In [44]:
data.tail()

,user_id,product_id,total_orders
13863744,206209,48697,1
13863745,206209,48742,2
13863746,206210,22802,97
13863747,206211,26834,89
13863748,206211,12590,77


## Query for Recommendations

We are now retrieving user factors for users that we have manually created before for testing purposes. Besides these users, we are adding a random existing user. We are also displaying these users so you can see what these factors look like.

In [45]:
user_ids = [206210, 206211, 103593]
user_factors = model.user_factors[user_to_index[user_ids]]

display(user_factors[1:])

array([[-0.313707  , -0.27664858, -1.2828674 ,  1.7485007 , -0.99290836,
         2.2764893 , -0.9494704 ,  1.1454551 , -2.0393448 , -1.6061454 ,
         0.5251323 , -1.3181114 ,  1.0427406 ,  1.8733109 , -2.549778  ,
         1.3321211 , -0.3133029 ,  2.208259  , -2.3020349 ,  0.5815489 ,
        -0.37377855,  0.7690419 , -0.40076098, -2.4527717 ,  1.2445227 ,
        -0.46194887, -2.7551808 , -1.1535347 ,  0.10361311,  2.6116688 ,
        -1.0924482 ,  1.6089348 , -2.4051344 , -2.931227  ,  1.8216416 ,
         1.0059888 ,  1.7596883 , -2.4735103 , -1.3774238 ,  1.2440904 ,
        -2.4833293 , -0.39517453, -0.00621076,  3.1343987 , -0.8446826 ,
        -1.5209638 ,  0.83655167,  1.7331113 , -0.35771912,  1.4926167 ,
         0.42455527,  0.6774385 , -0.63032067,  0.14995454, -1.2719532 ,
         1.0312878 ,  1.2454114 , -2.1102207 , -2.3025262 ,  0.55968195,
        -0.18068558,  0.99221903,  0.8804258 ,  0.9581545 , -0.7357688 ,
        -1.3392165 , -0.37945822,  1.0634457 , -0.1

### Model recommendations

We will now retrieve recommendations from our model directly, just to have these results as a baseline.

In [46]:
print("Model recommendations\n")

start_time = time.process_time()
recommendations0 = model.recommend(userid=user_ids[0], user_items=sparse_user_product[0])
recommendations1 = model.recommend(userid=user_ids[1], user_items=sparse_user_product[1])
recommendations2 = model.recommend(userid=user_ids[2], user_items=sparse_user_product[2])
print("Time needed for retrieving recommended products: " + str(time.process_time() - start_time) + ' seconds.\n')

print('\nRecommendations for person 0:')
for recommendation in recommendations0[0]:
    print(products_lookup[products_lookup.product_id == recommendation]['product_name'].values)

print('\nRecommendations for person 1:')
for recommendation in recommendations1[0]:
    print(products_lookup[products_lookup.product_id == recommendation]['product_name'].values)

print('\nRecommendations for person 2:')
for recommendation in recommendations2[0]:
    print(products_lookup[products_lookup.product_id == recommendation]['product_name'].values)

Model recommendations

Time needed for retrieving recommended products: 0.08775425699968764 seconds.


Recommendations for person 0:
['Sparkling Water']
['Mineral Water']
['Sparkling Natural Mineral Water']
['Soda']
['Spring Water']
['Smartwater']
['Zero Calorie Cola']
['Sparkling Mineral Water']
['Drinking Water']
['Distilled Water']

Recommendations for person 1:
['Baby Wipes Sensitive']
['Organic Garbanzo Beans']
['Ezekiel 4:9 Bread Organic Sprouted Whole Grain']
['Chocolate Ice Cream']
['YoBaby Peach Pear Yogurt']
['Free and Gentle High Efficiency Liquid Laundry Detergent']
['Baby Wash & Shampoo']
['No More Tears Baby Shampoo']
['White Buttermints']
['Eggo Pancakes Minis']

Recommendations for person 2:
['Organic Blackberries']
['Peach']
['Organic Bosc Pear']
['Organic Strawberries']
['Organic Blueberries']
['Red Plums']
['Clementines, Bag']
['Organic Bartlett Pear']
['Blood Oranges']
['Organic Fuji Apple']


### Query the index

Let's now query the index to check how quickly we retrieve results. Please note that query speed depends in part on your internet connection.

In [47]:
# Query by user factors
user_embeddings = user_factors.to_numpy()[:-1].tolist() if device == "cuda" else user_factors[:-1].tolist()

start_time = time.process_time()
query_results = index.query(queries=user_embeddings, top_k=10, include_metadata=True)
print("Time needed for retrieving recommended products using Pinecone: " + str(time.process_time() - start_time) + ' seconds.\n')

for _id, res in zip(user_ids, query_results.results):
    print(f'user_id={_id}')
    df = pd.DataFrame(
        {
            'products': [match.metadata['title'] for match in res.matches],
            'scores': [match.score for match in res.matches]
        }
    )
    print("Recommendation: ")
    display(df)
    print("Top buys from the past: ")
    display(products_bought_by_user_in_the_past(_id, top=15))

Time needed for retrieving recommended products using Pinecone: 0.016210021000006236 seconds.

user_id=206210
Recommendation: 


,products,scores
0,Mineral Water,0.927818
1,Zero Calorie Cola,0.660643
2,Orange & Lemon Flavor Variety Pack Sparkling F...,0.640357
3,Sparkling Water,0.617592
4,Extra Fancy Unsalted Mixed Nuts,0.596714
5,Popcorn,0.592487
6,Organic Variety Pack,0.586585
7,Drinking Water,0.585892
8,Tall Kitchen Bag With Febreze Odor Shield,0.569821
9,Milk Chocolate Almonds,0.561412


Top buys from the past: 


,product_id,product_name,total_orders
0,22802,Mineral Water,97


user_id=206211
Recommendation: 


,products,scores
0,Baby Wash & Shampoo,0.705169
1,No More Tears Baby Shampoo,0.675753
2,Baby Wipes Sensitive,0.563419
3,Size 6 Baby Dry Diapers,0.525253
4,Graduates Lil' Crunchies Mild Cheddar Corn Snacks,0.502849
5,Sensitive Infant Formula for Fussiness and Gas,0.494810
6,Size 5 Cruisers Diapers Super Pack,0.494335
7,Grow & Gain Chocolate Shake Nutritional Drink,0.494275
8,Stage 1 Newborn Hypoallergenic Liquid Detergent,0.492494
9,Strawberry Yogurt Melts,0.490631


Top buys from the past: 


,product_id,product_name,total_orders
0,26834,No More Tears Baby Shampoo,89
1,12590,Baby Wash & Shampoo,77


*Note* The inference using Pinecone is much faster compared to retrieving recommendations from a model directly. Please note that this result depends on your internet connection as well. 

All that’s left to do is surface these recommendations on the shopping site, or feed them into other applications.

## Clean up

Delete the index once you are sure that you do not want to use it anymore. Once it is deleted, you cannot reuse it.

In [48]:
pinecone.delete_index(index_name)

## Summary

In this example we used [Pinecone](https://www.pinecone.io/) to build and deploy a product recommendation engine that uses collaborative filtering, relatively quickly.

Once deployed, the product recommendation engine can index new data, retrieve recommendations in milliseconds, and send results to production applications.